In [1]:
import pandas as pd
import numba
import numpy as np
import pickle as pkl
import dask.dataframe as dd

In [2]:
@numba.vectorize(nopython=True)
def year_binner(year,val=10):
    return year - year%val

@numba.jit(nopython=True)
def groupby_numba(index, value, output):
    for i in range(index.shape[0]):
        output[index[i]] += value[i]

In [3]:
compounds=pd.read_pickle("/data/dharp/compounding/datasets/compounds.pkl")

In [20]:
compounds_test=compounds.head(100_000)

In [5]:
compounds.reset_index(inplace=True)
#compounds = dd.from_pandas(compounds, npartitions=30)
compounds.year=compounds.year.astype("int32")
compounds=compounds.query('1800 <= year <= 2010').copy()
compounds['time']=year_binner(compounds['year'].values,10)
#compounds = dd.from_pandas(compounds, npartitions=30)

#compounds=compounds.groupby(['modifier','head','context','time'])['count'].sum().to_frame()
#compounds.reset_index(inplace=True)
#compounds=compounds.loc[compounds['modifier'].isin(modifier_list) & compounds['head'].isin(head_list)]
compounds.info()
compounds.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 275119099 entries, 0 to 276118569
Data columns (total 6 columns):
modifier    object
head        object
context     object
year        int32
count       float64
time        int64
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 13.3+ GB


,modifier,head,context,year,count,time
0,a_noun,aaaaa_noun,a_noun,1877,1.0,1870
1,a_noun,aaaaa_noun,a_noun,1913,1.0,1910
2,a_noun,aaaaa_noun,a_noun,1925,3.0,1920
3,a_noun,aaaaa_noun,a_noun,1931,1.0,1930
4,a_noun,aaaaa_noun,a_noun,1948,1.0,1940


In [6]:
compounds_dd = dd.from_pandas(compounds, npartitions=30)
compounds_test=compounds_dd.groupby(['modifier','head','context','time'])['count'].sum().to_frame().compute()
compounds_test.reset_index()

,modifier,head,context,time,count
0,a_noun,aaaaa_noun,a_noun,1870,1.0
1,a_noun,aaaaa_noun,a_noun,1910,1.0
2,a_noun,aaaaa_noun,a_noun,1920,3.0
3,a_noun,aaaaa_noun,a_noun,1930,1.0
4,a_noun,aaaaa_noun,a_noun,1940,1.0
5,a_noun,aaaaa_noun,a_noun,1950,6.0
6,a_noun,aaaaa_noun,a_noun,1960,9.0
7,a_noun,aaaaa_noun,a_noun,1970,10.0
8,a_noun,aaaaa_noun,a_noun,1980,9.0
9,a_noun,aaaaa_noun,a_noun,1990,14.0


In [8]:
compounds_test.reset_index(inplace=True)
compounds_test.info()
compounds_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58475145 entries, 0 to 58475144
Data columns (total 5 columns):
modifier    object
head        object
context     object
time        int64
count       float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.2+ GB


,modifier,head,context,time,count
0,a_noun,aaaaa_noun,a_noun,1870,1.0
1,a_noun,aaaaa_noun,a_noun,1910,1.0
2,a_noun,aaaaa_noun,a_noun,1920,3.0
3,a_noun,aaaaa_noun,a_noun,1930,1.0
4,a_noun,aaaaa_noun,a_noun,1940,1.0


In [21]:
new_df=compounds_test.loc[compounds_test.groupby(['modifier','head','time'])['count'].transform('sum').gt(50)]


In [16]:
counts_df=compounds_test.groupby(['modifier','head','time'])['count'].sum()
counts_df

modifier       head          time
a_noun         aaaaa_noun    1870      1.0
                             1910      1.0
                             1920      3.0
                             1930      1.0
                             1940      1.0
                             1950      6.0
                             1960      9.0
                             1970     10.0
                             1980      9.0
                             1990     14.0
                             2000      8.0
               addison_noun  1960     28.0
                             1970     96.0
                             1980    206.0
                             1990    196.0
                             2000    222.0
               anti_noun     1950      4.0
                             1960     27.0
                             1970     30.0
                             1980     41.0
                             1990     67.0
                             2000     90.0
               archi

In [31]:
(new_df.shape[0]-compounds_test.shape[0])/compounds_test.shape[0]

-0.2137162036964594

In [30]:
new_df

,modifier,head,context,time,count
11,a_noun,addison_noun,boston_noun,2000,110.0
13,a_noun,addison_noun,publishing_noun,1970,48.0
14,a_noun,addison_noun,publishing_noun,1980,103.0
15,a_noun,addison_noun,publishing_noun,1990,98.0
16,a_noun,addison_noun,publishing_noun,2000,1.0
18,a_noun,addison_noun,wesley_noun,1970,48.0
19,a_noun,addison_noun,wesley_noun,1980,103.0
20,a_noun,addison_noun,wesley_noun,1990,98.0
21,a_noun,addison_noun,wesley_noun,2000,111.0
26,a_noun,anti_noun,b_noun,1990,67.0


In [26]:
head_list_reduced=new_df['head'].unique().tolist()

In [35]:
len(head_list_reduced)

74101

In [27]:
modifier_list_reduced=new_df['modifier'].unique().tolist()

In [34]:
len(modifier_list_reduced)

65175

In [28]:
pkl.dump(head_list_reduced,open('head_list_reduced.pkl','wb'))

In [29]:
pkl.dump(modifier_list_reduced,open('modifier_list_reduced.pkl','wb'))